In [ ]:
import pandas as pd
import numpy as np
import warnings
from datetime import datetime
from dateutil import relativedelta
from sklearn.preprocessing import MinMaxScaler
warnings.filterwarnings("ignore")

In [ ]:
sc      = MinMaxScaler()
df_09_1 = pd.read_csv('df_orig_09_1.csv')
df_09_2 = pd.read_csv('df_orig_09_2.csv')
df_09_3 = pd.read_csv('df_orig_09_3.csv')
df_09_4 = pd.read_csv('df_orig_09_4.csv')
df_10_1 = pd.read_csv('df_orig_10_1.csv')
df_10_2 = pd.read_csv('df_orig_10_2.csv')
df_10_3 = pd.read_csv('df_orig_10_3.csv')
df_10_4 = pd.read_csv('df_orig_10_4.csv')
df      = pd.concat([df_09_1, df_09_2, df_09_3, df_09_4, df_10_1, df_10_2, df_10_3, df_10_4])
df.reset_index(drop = True, inplace = True)

In [ ]:
df.drop(['dt_first_pi', 'dt_matr', 'cd_msa', 'cltv', 'st',
         'servicer_name', 'prod_type', 'ppmt_pnlty'], axis = 1, inplace = True)

In [ ]:
df.drop(df[df['flag_fthb'] == 'Other'].index, inplace = True)
df.drop(df[df['seller_name'] == 'Other sellers'].index, inplace = True)
df.reset_index(drop = True, inplace = True)

In [ ]:
df['id'] = np.arange(1, len(df) + 1)

In [ ]:
df['fico'] = df['fico'].astype(int)

In [ ]:
df['flag_fthb'] = df['flag_fthb'].where(df['flag_fthb'] != 'Y', '1')
df['flag_fthb'] = df['flag_fthb'].where(df['flag_fthb'] != 'N', '0')
df['flag_fthb'] = df['flag_fthb'].astype(int)
df.rename(columns = {'flag_fthb':'if_fthb'}, inplace = True)

In [ ]:
df['mi_pct'] = df['mi_pct'].astype(int)

In [ ]:
df['cnt_units'] = df['cnt_units'].astype(int)

In [ ]:
df['occpy_sts'] = df['occpy_sts'].where(df['occpy_sts'] != 'P', '1')
df['occpy_sts'] = df['occpy_sts'].where(df['occpy_sts'] != 'S', '0')
df['occpy_sts'] = df['occpy_sts'].where(df['occpy_sts'] != 'I', '0')
df['occpy_sts'] = df['occpy_sts'].astype(int)
df.rename(columns = {'occpy_sts':'if_prim_res'}, inplace = True)

In [ ]:
df['dti'] = df['dti'].astype(int)

In [ ]:
df.rename(columns = {'orig_upb':'upb'}, inplace = True)

In [ ]:
df['ltv'] = df['ltv'].astype(int)

In [ ]:
df['channel'] = df['channel'].where(df['channel'] != 'C', '1')
df['channel'] = df['channel'].where(df['channel'] != 'R', '0')
df['channel'] = df['channel'].where(df['channel'] != 'B', '0')
df['channel'] = df['channel'].where(df['channel'] != 'T', '0')
df['channel'] = df['channel'].astype(int)
df.rename(columns = {'channel':'if_corr'}, inplace = True)

In [ ]:
df['prop_type'] = df['prop_type'].where(df['prop_type'] != 'SF', '1')
df['prop_type'] = df['prop_type'].where(df['prop_type'] != 'CO', '0')
df['prop_type'] = df['prop_type'].where(df['prop_type'] != 'PU', '0')
df['prop_type'] = df['prop_type'].where(df['prop_type'] != 'MH', '0')
df['prop_type'] = df['prop_type'].where(df['prop_type'] != 'CP', '0')
df['prop_type'] = df['prop_type'].astype(int)
df.rename(columns = {'prop_type':'if_sf'}, inplace = True)

In [ ]:
df['zipcode'] = df['zipcode'].astype(str)
df['length'] = df.zipcode.str.len()
df = df[df.length > 6]
df['area'] = df['zipcode'].apply(lambda x: x[0:2]).astype(int)
df.drop(['length', 'zipcode'], axis = 1, inplace = True)
df.reset_index(drop = True, inplace = True)

In [ ]:
df['loan_purpose'] = df['loan_purpose'].where(df['loan_purpose'] != 'P', '1')
df['loan_purpose'] = df['loan_purpose'].where(df['loan_purpose'] != 'N', '0')
df['loan_purpose'] = df['loan_purpose'].where(df['loan_purpose'] != 'C', '0')
df['loan_purpose'] = df['loan_purpose'].astype(int)
df.rename(columns = {'loan_purpose':'if_purc'}, inplace = True)

In [ ]:
df.rename(columns = {'orig_loan_term':'loan_term'}, inplace = True)

In [ ]:
df['cnt_borr'] = df['cnt_borr'].astype(int)

In [ ]:
df.rename(columns = {'seller_name':'provider'}, inplace = True)

In [ ]:
df['flag_sc'] = df['flag_sc'].where(df['flag_sc'] != 'Y', '1')
df['flag_sc'] = df['flag_sc'].where(df['flag_sc'] != 'N', '0')
df['flag_sc'] = df['flag_sc'].astype(int)
df.rename(columns = {'flag_sc':'if_sc'}, inplace = True)

In [ ]:
df['Default'] = df['Default'].where(df['Default'] != 'YES', '1')
df['Default'] = df['Default'].where(df['Default'] != 'NO', '0')
df['Default'] = df['Default'].astype(int)
df.rename(columns = {'Default':'default'}, inplace = True)

In [ ]:
df_d = df[df['svcg_cycle'] != 'N_A']
df_d.reset_index(drop = True, inplace = True)
df_d['svcg_cycle'] = df_d['svcg_cycle'].apply(lambda x: x[0:6])

def f(x):
    r = relativedelta.relativedelta(datetime.strptime(x, "%Y%m"), datetime.strptime('200902', "%Y%m"))
    return r.months + (12 * r.years);

df_d['d_timer'] = df_d.apply(lambda x: f(x['svcg_cycle']), axis = 1)
drop_list = [column for column in df_d.columns if column not in ['id_loan', 'd_timer']]
df_d.drop(drop_list, axis = 1, inplace = True)
df = pd.merge(df, df_d, on = 'id_loan', how = 'outer')
df['d_timer'] = df['d_timer'].where(df['d_timer'].isnull() == False, 1000)
df = df[df['d_timer'] >= 35]
df.reset_index(drop = True, inplace = True)

In [ ]:
cols = df.columns
cols = cols.drop(['id', 'id_loan', 'year', 'month', 'area', 'provider'])
cols = ['id', 'id_loan', 'year', 'month', 'area', 'provider'] + list(cols)
df   = df[cols]
df.sort_values(by = ['id'], inplace = True, ignore_index = True)

In [ ]:
df_mp_09_1 = pd.read_csv('df_mp_09_1.csv')
df_mp_09_2 = pd.read_csv('df_mp_09_2.csv')
df_mp_09_3 = pd.read_csv('df_mp_09_3.csv')
df_mp_09_4 = pd.read_csv('df_mp_09_4.csv')
df_mp_10_1 = pd.read_csv('df_mp_10_1.csv')
df_mp_10_2 = pd.read_csv('df_mp_10_2.csv')
df_mp_10_3 = pd.read_csv('df_mp_10_3.csv')
df_mp_10_4 = pd.read_csv('df_mp_10_4.csv')
df_mp      = pd.concat([df_mp_09_1, df_mp_09_2, df_mp_09_3, df_mp_09_4, df_mp_10_1, df_mp_10_2, df_mp_10_3, df_mp_10_4])
df_mp      = df_mp[df_mp['id_loan'].isin(list(df['id_loan']))]
df_mp      = df_mp[df_mp['loan_age'] < 60]
df_mp.drop_duplicates(subset = ['id_loan', 'loan_age'], keep = 'first', inplace = True)

In [ ]:
df_mp['delq_sts'] = df_mp['delq_sts'].where(df_mp['delq_sts'] == '0', '1')
df_mp['delq_sts'] = df_mp['delq_sts'].astype(int)
df_mp.rename(columns = {'delq_sts':'if_delq_sts'}, inplace = True)

In [ ]:
l1 = []

for i in range(0, 60):
    l2 = list(set(df_mp['id_loan'].unique()) - set(df_mp[df_mp['loan_age'] == i]['id_loan'].unique()))
    l1 = list(set(l1 + l2))

l3 = list(set(df_mp['svcg_cycle'].unique()).difference(set([200902, 200903, 200904, 200905, 200906, 200907, 200908,
                                                            200909, 200910, 200911, 200912, 201001, 201002, 201003,
                                                            201004, 201005, 201006, 201007, 201008, 201009, 201010,
                                                            201011, 201012, 201101, 201102, 201103, 201104, 201105,
                                                            201106, 201107, 201108, 201109, 201110, 201111, 201112,
                                                            201201, 201202, 201203, 201204, 201205, 201206, 201207,
                                                            201208, 201209, 201210, 201211, 201212, 201301, 201302,
                                                            201303, 201304, 201305, 201306, 201307, 201308, 201309,
                                                            201310, 201311, 201312, 201401, 201402, 201403, 201404,
                                                            201405, 201406, 201407, 201408, 201409, 201410, 201411,
                                                            201412, 201501, 201502, 201503, 201504, 201505, 201506,
                                                            201507, 201508, 201509, 201510, 201511, 201512])))
l4 = list(df_mp[df_mp['svcg_cycle'].isin(l3)]['id_loan'].unique())
ll = list(set(l1 + l4))
df_mp = df_mp[~df_mp['id_loan'].isin(ll)]

In [ ]:
df_mp['year']  = df_mp['svcg_cycle'].astype(str).apply(lambda x: x[0:4]).astype(int)
df_mp['month'] = df_mp['svcg_cycle'].astype(str).apply(lambda x: x[4:6]).astype(int)
df_mp = df_mp[df_mp['svcg_cycle'].isin([201201, 201202, 201203, 201204, 201205, 201206,
                                        201207, 201208, 201209, 201210, 201211, 201212,
                                        201301, 201302, 201303, 201304, 201305, 201306,
                                        201307, 201308, 201309, 201310, 201311, 201312])]
df_mp['counter'] = (df_mp['year'] - 2012) * 12 + df_mp['month']
df_mp.reset_index(drop = True, inplace = True)
df_mp.to_csv('df_mp_tot.csv', index = False)

In [ ]:
df = df[df['id_loan'].isin(list(df_mp['id_loan'].unique()))]
df.reset_index(drop = True, inplace = True)
df.to_csv('df_orig_tot.csv', index = False)

In [ ]:
df_tr_new = df_train
df_tr_new.sort_values(by = ['id'], inplace = True, ignore_index = True)
df_tr_temp_1 = df_tr_new.loc[:, df_tr_new.columns.isin(['id', 'id_loan', 'year', 'month', 'area', 'provider', 'svcg_cycle',
                                                        'default', 'd_timer'])]
df_tr_temp_2 = df_tr_new.loc[:, ~df_tr_new.columns.isin(['id', 'id_loan', 'year', 'month', 'area', 'provider', 'svcg_cycle',
                                                         'default', 'd_timer'])]
df_tr_temp_2_sc = pd.DataFrame(sc.fit_transform(df_tr_temp_2), columns = df_tr_temp_2.columns)
df_tr_t = pd.concat([df_tr_temp_1, df_tr_temp_2_sc], axis = 1)
df_tr_t = (df_tr_t[['id', 'id_loan', 'year', 'month', 'area', 'provider'] + list(df_tr_temp_2_sc.columns) +
                   ['svcg_cycle', 'default', 'd_timer']])
df_tr_t.to_csv('df_orig_tr_sc.csv', index = False)

In [ ]:
df_te_new = df_test
df_te_new.sort_values(by = ['id'], inplace = True, ignore_index = True)
df_te_temp_1 = df_te_new.loc[:, df_te_new.columns.isin(['id', 'id_loan', 'year', 'month', 'area', 'provider', 'svcg_cycle',
                                                        'default', 'd_timer'])]
df_te_temp_2 = df_te_new.loc[:, ~df_te_new.columns.isin(['id', 'id_loan', 'year', 'month', 'area', 'provider', 'svcg_cycle',
                                                         'default', 'd_timer'])]
df_te_temp_2_sc = pd.DataFrame(sc.transform(df_te_temp_2), columns = df_te_temp_2.columns)
df_te_t = pd.concat([df_te_temp_1, df_te_temp_2_sc], axis = 1)
df_te_t = (df_te_t[['id', 'id_loan', 'year', 'month', 'area', 'provider'] + list(df_te_temp_2_sc.columns) +
                   ['svcg_cycle', 'default', 'd_timer']])
df_te_t.to_csv('df_orig_te_sc.csv', index = False)

In [ ]:
df_tr_mp_new = df_mp[df_mp['id_loan'].isin(list(df_tr_t['id_loan'].unique()))]
df_tr_mp_new.reset_index(drop = True, inplace = True)
df_tr_mp_temp_1 = df_tr_mp_new.loc[:, df_tr_mp_new.columns.isin(['id_loan', 'year', 'month', 'svcg_cycle', 'loan_age',
                                                                 'counter'])]
df_tr_mp_temp_2 = df_tr_mp_new.loc[:, ~df_tr_mp_new.columns.isin(['id_loan', 'year', 'month', 'svcg_cycle', 'loan_age',
                                                                  'counter'])]
df_tr_mp_temp_2_sc = pd.DataFrame(sc.fit_transform(df_tr_mp_temp_2), columns = df_tr_mp_temp_2.columns)
df_tr_mp_t = pd.concat([df_tr_mp_temp_1, df_tr_mp_temp_2_sc], axis = 1)
df_tr_mp_t.to_csv('df_mp_tr_sc.csv', index = False)

In [ ]:
df_te_mp_new = df_mp[df_mp['id_loan'].isin(list(df_te_t['id_loan'].unique()))]
df_te_mp_new.reset_index(drop = True, inplace = True)
df_te_mp_temp_1 = df_te_mp_new.loc[:, df_te_mp_new.columns.isin(['id_loan', 'year', 'month', 'svcg_cycle', 'loan_age',
                                                                 'counter'])]
df_te_mp_temp_2 = df_te_mp_new.loc[:, ~df_te_mp_new.columns.isin(['id_loan', 'year', 'month', 'svcg_cycle', 'loan_age',
                                                                  'counter'])]
df_te_mp_temp_2_sc = pd.DataFrame(sc.transform(df_te_mp_temp_2), columns = df_te_mp_temp_2.columns)
df_te_mp_t = pd.concat([df_te_mp_temp_1, df_te_mp_temp_2_sc], axis = 1)
df_te_mp_t.to_csv('df_mp_te_sc.csv', index = False)